In [1]:
import geemap,ee
import pandas as pd
import os

ee.Initialize()

roi = ee.FeatureCollection("users/yehuigeo/poyanghu")
shp = ee.Feature(roi.first()).geometry()

In [2]:
def cloudfunction_ST2(image):
    # use add the cloud likelihood band to the image
    qa = image.select("QA60")
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000)\
        .select("B.*")\
        .copyProperties(image, ["system:index","system:time_start"])

def NMDI(img):
    nmdi = img.expression(
        '(NIR-(SWIR1-SWIR2))/(NIR+(SWIR1-SWIR2))', {
          'NIR': img.select('B8'),
          'SWIR1': img.select('B11'),
          'SWIR2':img.select('B12')}).rename('NMDI')
    return nmdi

def NDVI(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return ndvi

In [4]:
bands = ['NMDI']

s2m = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
            .filter(ee.Filter.calendarRange(2017,2021,'year')).filter(ee.Filter.calendarRange(4,4+1,'month'))\
            .filterBounds(shp)\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',25))\
            .sort('CLOUDY_PIXEL_PERCENTAGE', False)\
            .map(cloudfunction_ST2)\
            .mosaic()

s2m = s2m.addBands(NDVI(s2m))
s2m = s2m.addBands(NMDI(s2m))
s2m = s2m.where(s2m.gt(1),1).where(s2m.lt(0),0)

from user_gee import clip_dow_merge
clip_dow_merge(shp, s2m.select(bands), r'C:\Users\xiong\Downloads\data\NMDI4_1', 20, 10, 'EPSG:3857')

影像切割完毕！！！
需要下载的影像数: 11

Generating URL ...
Generating URL ...
Please wait ...
Please wait ...
Data downloaded to C:\Users\xiong\Downloads\data\NMDI4_1_mk\0.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\xiong\Downloads\data\NMDI4_1_mk\1.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\xiong\Downloads\data\NMDI4_1_mk\2.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\xiong\Downloads\data\NMDI4_1_mk\3.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\xiong\Downloads\data\NMDI4_1_mk\4.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\xiong\Downloads\data\NMDI4_1_mk\5.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\xiong\Downloads\data\NMDI4_1_mk\6.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\xiong\Downloads\data\NMDI4_1_mk\7.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\xiong\Downloads\data\NMDI4_1_mk\8.tif
Generating URL ...
Please wait .